In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Estilo visual moderno
sns.set(style="whitegrid", palette="muted", font_scale=1.2)

# === 1. Carrega o CSV com métricas por carteira/token ===
df = pd.read_csv('../data/graphs/all_metrics.csv')
df['date'] = pd.to_datetime(df['date'])

# === 2. Agrupa por dia/token e calcula médias ===
df_grouped = df.groupby(['date', 'token']).agg({
    'in_degree': 'mean',
    'out_degree': 'mean',
    'pagerank': 'mean',
    'degree_centrality': 'mean'
}).reset_index()

# === 3. Função de visualização ===
def plot_metric(df_grouped, metric, event_date=None, title=None):
    plt.figure(figsize=(12, 6))
    
    for token in df_grouped['token'].unique():
        df_token = df_grouped[df_grouped['token'] == token]
        plt.plot(df_token['date'], df_token[metric], label=token)

    if event_date:
        plt.axvline(pd.to_datetime(event_date), color='red', linestyle='--', label='Event')

    plt.title(title or f'{metric.title()} over Time')
    plt.xlabel('Date')
    plt.ylabel(metric.replace('_', ' ').title())
    plt.legend(title='Token')
    plt.tight_layout()
    plt.show()

# === 4. Plotar o gráfico desejado ===
plot_metric(
    df_grouped,
    metric='pagerank',  # pode trocar por: 'in_degree', 'out_degree', 'degree_centrality'
    event_date='2022-05-07',
    title='Mean Pagerank per Token Over Time'
)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Estilo bonito
sns.set(style="whitegrid", palette="muted", font_scale=1.2)

# === 1. Carrega os dados ===
df = pd.read_csv('../data/graphs/all_metrics.csv')
df['date'] = pd.to_datetime(df['date'])

# === 2. Defina o endereço da carteira que deseja analisar ===
carteira_alvo = df['address'].value_counts().idxmax()  # ou defina manualmente: '0x...'

# === 3. Filtra dados da carteira ao longo do tempo ===
df_carteira = df[df['address'] == carteira_alvo]

# === 4. Plota a evolução do pagerank ===
plt.figure(figsize=(12, 6))
plt.plot(df_carteira['date'], df_carteira['pagerank'], marker='o')
plt.axvline(pd.to_datetime('2022-05-07'), color='red', linestyle='--', label='UST Depeg')
plt.title(f'Pagerank da carteira {carteira_alvo[:8]}... ao longo do tempo')
plt.xlabel('Data')
plt.ylabel('Pagerank')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Estilo visual moderno
sns.set(style="whitegrid", palette="muted", font_scale=1.2)

# 1. Carrega os dados
df = pd.read_csv('../data/graphs/all_metrics.csv')
df['date'] = pd.to_datetime(df['date'])

# 2. Calcula a média de pagerank por carteira (independente da data)
top_addresses = df.groupby('address')['pagerank'].mean().nlargest(5).index.tolist()

# 3. Filtra o dataframe para essas 5 carteiras
df_top = df[df['address'].isin(top_addresses)]


# 4. Plota a evolução do pagerank dessas carteiras
plt.figure(figsize=(14, 6))

for addr in top_addresses:
    df_addr = df_top[df_top['address'] == addr]
    plt.plot(df_addr['date'], df_addr['pagerank'], marker='o', label=f'{addr[:8]}...')

# Linha do evento de depeg
plt.axvline(pd.to_datetime('2022-05-07'), color='red', linestyle='--', label='UST Depeg')

# Títulos e legenda
plt.title('Evolução do Pagerank das Top 5 carteiras')
plt.xlabel('Data')
plt.ylabel('Pagerank')
plt.legend(title='Carteira')
plt.tight_layout()
plt.show()



In [ ]:
import pandas as pd

# Carregar dados
df = pd.read_csv('../data/graphs/all_metrics.csv')
df['date'] = pd.to_datetime(df['date'])

# Separar antes e depois do evento
before = df[df['date'] < '2022-05-07']
after = df[df['date'] >= '2022-05-07']

# Carteiras que usavam UST antes do colapso
ust_before = before[before['token'] == 'UST']['address'].unique()

# Carteiras que usaram UST depois do colapso
ust_after = after[after['token'] == 'UST']['address'].unique()

# Carteiras que SUMIRAM do UST depois
exited_ust = set(ust_before) - set(ust_after)

# Agora vamos ver em quais tokens essas carteiras apareceram depois
after_switch = after[after['address'].isin(exited_ust)]

# Tokens que essas carteiras passaram a usar
migrated = after_switch.groupby('token')['address'].nunique().sort_values(ascending=False)

# Mostrar top tokens para onde as carteiras “fugiram”
print("Tokens de destino das carteiras que abandonaram a UST:")
print(migrated)
